In [1]:
import numpy as np
import pandas as pd

In [2]:
data = [
    ("A", "I", 1),
    ("A", "II", 2),
    ("A", "III", 3),
    ("A", "IV", 4),
    ("B", "I", 3),
    ("B", "II", 2),
    ("B", "III", 1),
    ("B", "IV", 4),
    ("C", "I", 1),
    ("C", "II", 2),
    ("C", "III", 4),
    ("C", "IV", 4)
]

data4 = [
    ("A", "I", 1),
    ("A", "II", 2),
    ("A", "III", 3),
    ("B", "I", 3),
    ("B", "II", 2),
    ("B", "III", 1),
    ("B", "IV", 4),
    ("C", "I", 1),
    ("C", "II", 2),
    ("C", "III", 4),
    ("C", "IV", 4)
]

# Voters	Ranking
data2 = [
    (30, ["A", "B", "C"]),
    ( 1, ["A", "C", "B"]),
    (29, ["B", "A", "C"]),
    (10, ["B", "C", "A"]),
    (10, ["C", "A", "B"]),
    ( 1, ["C", "B", "A"])
]

data3 = [
    (7, ["A", "B", "C", "D"]),
    (5, ["B", "C", "D", "A"]),
    (4, ["D", "B", "C", "A"]),
    (3, ["C", "D", "A", "B"])
]

data5 = [
    (3, ["A", "B", "C"]),
    (2, ["B", "A", "C"]),
    (5, ["C", "A", "B"])
]

In [3]:
df = pd.DataFrame(data, columns=["voter", "candidate", "rank"])
df4 = pd.DataFrame(data4, columns=["voter", "candidate", "rank"])
df2 = pd.DataFrame(data2, columns=["voters", "rank"])
df3 = pd.DataFrame(data3, columns=["voters", "rank"])
df5 = pd.DataFrame(data5, columns=["voters", "rank"])

In [4]:
df = pd.DataFrame([
    ("Memphis", "M>N>C>K", 42),
    ("Nashville", "N>C>K>M", 26),
    ("Chattanooga", "C>K>N>M", 15),
    ("Knoxville", "K>C>N>M", 17)
], columns=["voter", "rank", "voters"])
df

,voter,rank,voters
0,Memphis,M>N>C>K,42
1,Nashville,N>C>K>M,26
2,Chattanooga,C>K>N>M,15
3,Knoxville,K>C>N>M,17


In [12]:
import numpy as np
from comchoice.aggregate.__aggregate import __aggregate
from comchoice.aggregate.__set_rank import __set_rank
from comchoice.aggregate.__set_voters import __set_voters
from comchoice.aggregate.__transform import __transform


def judgment(
    df,
    alternative="alternative",
    ballot_method="rank",
    column="rank",
    delimiter=">",
    method="typical",
    ratings=None,
    show_rank=True,
    source="rank",
    voter="voter",
    voters="voters",
    e=0
):
    if voters in list(df):
        df = __transform(
            df, 
            ballot=ballot_method,
            unique_id=True
        )
    else:
        df[voters] = 1
    
    ascending = True
    if source == "rank":
        jdgm = []
        for _alternative, tmp in df.groupby("_id"):
            output = []
            for i, items in tmp.iterrows():
                output += [items[column]] * items[voters]

            jdgm.append([_alternative, np.median(output)])

        jdgm = pd.DataFrame(jdgm, columns=["_id", "alpha"])
        jdgm = pd.merge(df, jdgm, on="_id")
        
    elif source == "score":
        jdgm = __aggregate(df, groupby=[alternative], aggregation="median", column=column)\
            .rename(columns={column: "alpha"})
        jdgm = pd.merge(df, jdgm, on=alternative)

        ascending = False
        
    display(jdgm)

    jdgm["p" if ascending else "q"] = jdgm[column] < np.floor(jdgm["alpha"]) # Rate higher than median
    jdgm["q" if ascending else "p"] = jdgm[column] > np.floor(jdgm["alpha"]) # Rate lower than median

    for col in ["p", "q"]:
        jdgm[col] = jdgm[col].astype(int)

    jdgm = jdgm.groupby(alternative).agg({"alpha": "mean", "p": "mean", "q": "mean"}).reset_index()
    
    if method == "typical":
        jdgm["value"] = jdgm["alpha"] + jdgm["p"] - jdgm["q"]
        
    elif method == "usual":
        jdgm["value"] = jdgm["alpha"] + 0.5 * (jdgm["p"] - jdgm["q"]) / (1 - jdgm["p"] - jdgm["q"])

    elif method == "central":
        jdgm["value"] = jdgm["alpha"] + 0.5 * (jdgm["p"] - jdgm["q"]) / (jdgm["p"] + jdgm["q"] + e)
        
    elif method == "bucklin":
        jdgm["value"] = jdgm["alpha"] - jdgm["q"]
        
    elif method == "majority":
        jdgm["value"] = jdgm.apply(lambda x: x["p"] if (x["p"] > x["q"]) else -x["q"], axis=1)
        
    if show_rank:
        jdgm = __set_rank(jdgm, ascending=False)

    return jdgm

In [6]:
__set_rank

<function comchoice.aggregate.__set_rank.__set_rank(df, ascending=False) -> pandas.core.frame.DataFrame>

In [13]:
#voters	their vote
# 49	A=100	B=52	C=0	D=0
# 1	A=50	B=51	C=100	D=0
# 49	A=49	B=0	C=0	D=100

df2 = pd.DataFrame([(49, "A=100,B=52,C=0,D=0"),
( 1, "A=50,B=51,C=100,D=0"),
(49, "A=49,B=0,C=0,D=100")],
             columns=["voters", "ballot"])
display(df2)

judgment(
    df2,
    column="score",
    source="score",
    method="majority",
    ballot_method="score"
)

,voters,ballot
0,49,"A=100,B=52,C=0,D=0"
1,1,"A=50,B=51,C=100,D=0"
2,49,"A=49,B=0,C=0,D=100"


,voters,_id,alternative,score,alpha
0,49,0,A,100.0,50.0
1,1,1,A,50.0,50.0
2,49,2,A,49.0,50.0
3,49,0,B,52.0,51.0
4,1,1,B,51.0,51.0
5,49,2,B,0.0,51.0
6,49,0,C,0.0,0.0
7,1,1,C,100.0,0.0
8,49,2,C,0.0,0.0
9,49,0,D,0.0,0.0


,alternative,alpha,p,q,value,rank
2,C,0.0,0.333333,0.000000,0.333333,1
3,D,0.0,0.333333,0.000000,0.333333,1
0,A,50.0,0.333333,0.333333,-0.333333,3
1,B,51.0,0.333333,0.333333,-0.333333,3


In [8]:
# tmp = df2.copy()
# tmp["alternative"] = tmp["ballot"].str.split(",")
# tmp = tmp.explode("alternative")
# tmp[["alternative", "score"]] = tmp["alternative"].str.split("=", n=1, expand=True)
# tmp[[voters, alternative, score]]

In [9]:
df1 = pd.DataFrame([
    ("A", 5),
    ("A", 3),
    ("A", 3),
    ("A", 2),
    ("A", 2),
    ("B", 5),
    ("B", 4),
    ("B", 1),
    ("B", 1),
    ("B", 2)
], columns=["alternative", "score"])

judgment(
    df1,
    column="score",
    source="score",
    method="usual"
)

,alternative,score,voters,alpha
0,A,5,1,3.0
1,A,3,1,3.0
2,A,3,1,3.0
3,A,2,1,3.0
4,A,2,1,3.0
5,B,5,1,2.0
6,B,4,1,2.0
7,B,1,1,2.0
8,B,1,1,2.0
9,B,2,1,2.0


,alternative,alpha,p,q,value,rank
0,A,3.0,0.2,0.4,2.75,1
1,B,2.0,0.4,0.4,2.00,2


In [10]:
judgment(
    df,
    method="typical"
)

,voter,alternative,voters,_id,rank,alpha
0,Memphis,M,42,0,1,2.5
1,Memphis,N,42,0,2,2.5
2,Memphis,C,42,0,3,2.5
3,Memphis,K,42,0,4,2.5
4,Nashville,N,26,1,1,2.5
5,Nashville,C,26,1,2,2.5
6,Nashville,K,26,1,3,2.5
7,Nashville,M,26,1,4,2.5
8,Chattanooga,C,15,2,1,2.5
9,Chattanooga,K,15,2,2,2.5


,alternative,alpha,p,q,value,rank
0,C,2.5,0.25,0.25,2.50,1
1,K,2.5,0.25,0.50,2.25,2
3,N,2.5,0.25,0.50,2.25,2
2,M,2.5,0.25,0.75,2.00,4
